In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-movies-dataset-2023-930k-movies/TMDB_movie_dataset_v11.csv


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
moive=pd.read_csv(r'/kaggle/input/tmdb-movies-dataset-2023-930k-movies/TMDB_movie_dataset_v11.csv')[:20001]

In [5]:
moive.shape

(20001, 24)

In [6]:
moive.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [7]:
col=['id','title','overview','imdb_id','poster_path','tagline','genres','keywords']
for feature in col:
  moive[feature]=moive[feature].fillna('')

In [8]:
moive_df=moive[col]
moive_df.isnull().sum()



id             0
title          0
overview       0
imdb_id        0
poster_path    0
tagline        0
genres         0
keywords       0
dtype: int64

In [9]:
moive['tags']=moive['overview']+moive['genres']+moive['keywords']+moive['tagline']

In [10]:
moive.head(1)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,tags
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc...","Cobb, a skilled thief who commits corporate es..."


In [11]:
moive['tags'][0]

'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.Action, Science Fiction, Adventurerescue, mission, dream, airplane, paris, france, virtual reality, kidnapping, philosophy, spy, allegory, manipulation, car crash, heist, memory, architecture, los angeles, california, dream world, subconsciousYour mind is the scene of the crime.'

In [12]:
!pip install neattext

In [13]:
import neattext as nfx

In [14]:
moive['tags']=moive['tags'].apply(lambda x:x.lower())
moive['tags']=moive['tags'].apply(lambda x:x.replace('|',' '))
moive['tags']=moive['tags'].apply(lambda x:x.replace(',',' '))
moive['tags']=moive['tags'].apply(lambda x:nfx.remove_stopwords(x))
moive['tags']=moive['tags'].apply(lambda x:nfx.remove_special_characters(x))

In [15]:
moive['tags'][300]

'cartoonist teams ace reporter law enforcement officer track elusive serial killercrime drama mystery thrillernewspaper journalist california based novel book police san francisco california killing detective 1970s investigation victim code murder serial killer crime scene reporter whodunit alcoholic young man fourth july newspaper article cartoonist 1960s zodiac killer bartheres way lose life killer'

In [16]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [17]:
import nltk
nltk.download('punkt_tab') # Changed from 'punkt' to 'punkt_tab'
from nltk.tokenize import word_tokenize
moive['tags'] = moive['tags'].astype(str)
moive['tokens'] = moive['tags'].apply(word_tokenize)

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [18]:
moive['tokens']=moive['tokens'].apply(lambda x:[stemmer.stem(i) for i in x])

In [19]:
moive["tags_tokens"]=moive['tokens'].apply(lambda x:' '.join(x))

In [20]:
moive["tags_tokens"]

0        cobb skill thief commit corpor espionag infilt...
1        adventur group explor use newli discov wormhol...
2        batman rais stake war crime help lt jim gordon...
3        22nd centuri parapleg marin dispatch moon pand...
4        unexpect enemi emerg threaten global safeti se...
                               ...                        
19996    ana year old girl arriv mother isol beach inte...
19997    middleclass parisian suburb blanch la offic wo...
19998    daughter crimin boyfriend increas threat famil...
19999    follow aftermath horrif accid woman voluntaril...
20000    quaint villag inhabit descend italian settler ...
Name: tags_tokens, Length: 20001, dtype: object

In [21]:
moive_feautures=moive[['id','title','tags_tokens','poster_path']]
moive_feautures

,id,title,tags_tokens,poster_path
0,27205,Inception,cobb skill thief commit corpor espionag infilt...,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg
1,157336,Interstellar,adventur group explor use newli discov wormhol...,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg
2,155,The Dark Knight,batman rais stake war crime help lt jim gordon...,/qJ2tW6WMUDux911r6m7haRef0WH.jpg
3,19995,Avatar,22nd centuri parapleg marin dispatch moon pand...,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg
4,24428,The Avengers,unexpect enemi emerg threaten global safeti se...,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg
...,...,...,...,...
19996,265339,Ana and Bruno,ana year old girl arriv mother isol beach inte...,/njGHy8ZKtREjWbZnGBN5rwpxFzL.jpg
19997,64544,Boyfriends and Girlfriends,middleclass parisian suburb blanch la offic wo...,/29znfm8kEe1htpDuCrsr7QyBrg4.jpg
19998,346658,The Oath,daughter crimin boyfriend increas threat famil...,/aNYmpfY4L9EHlgrfvDTWYZmx6FA.jpg
19999,897424,Dark Cloud,follow aftermath horrif accid woman voluntaril...,/q1ngkvPCnOYKeyNZjg8QqvaSnuZ.jpg


In [78]:
 from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_features=10000)
vector=tfidf.fit_transform(moive_feautures['tags_tokens']).toarray()

In [79]:
vector.shape

(20001, 10000)

In [80]:
feature_names = tfidf.get_feature_names_out()
feature_names


array(['000', '007', '10', ..., 'zone', 'zoo', 'zookeep'], dtype=object)

In [81]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)

In [82]:
similarity.shape

(20001, 20001)

In [83]:
moive_list=sorted(enumerate(similarity[0]),reverse=True,key=lambda x:x[1])[1:11]

In [84]:
moive_list
for x in moive_list:
  print(moive_feautures.iloc[x[0]].title)

The City of Lost Children
The Thirteenth Floor
Henry's Crime
Total Recall
Little Nemo: Adventures in Slumberland
Cypher
Charlie's Angels: Full Throttle
Heat
Strange Days
Total Recall


In [85]:
from IPython.display import Image, display
def recommendation(moive):
  moive_index=moive_feautures[moive_feautures['title']==moive].index[0]
  moive_list=sorted(enumerate(similarity[moive_index]),reverse=True,key=lambda x:x[1])[0:6]


  for x in moive_list:
      print(moive_feautures.iloc[x[0]].title)
      display(Image(url='https://image.tmdb.org/t/p/w500/'+str(moive_feautures.iloc[x[0]].poster_path)))

In [86]:
 !pip install requests

In [87]:
import requests
image=requests.get(url='https://image.tmdb.org/t/p/w500//qhPtAc1TKbMPqNvcdXSOn9Bn7hZ.jpg')
with open('poster.jpg', 'wb') as f:
    f.write(image.content)

In [96]:
recommendation("Spider-Man: Into the Spider-Verse")

Spider-Man: Into the Spider-Verse


Spider-Man: Across the Spider-Verse


First Match


Miles Ahead


Spider-Man


The Flash


In [51]:
import pickle
pickle.dump(moive_feautures,open('moive_dict.pkl','wb'))
pickle.dump(similarity,open('similarity_new.pkl','wb'))
pickle.dump(moive,open("moive_file.pkl","wb"))